In [1]:
# python
from collections import defaultdict
from functools import partial
import nltk
import random
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from collections import defaultdict
from nltk import FreqDist
from nltk import ConditionalFreqDist
import re
from nltk import ngrams


## Preprocessing the text

In [2]:

import re

def preprocess_text(file_path):
    # Read the text from the file from line 174 to line 37247
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()[173:37246]  # Read from line 174 to line 37247
        text = ''.join(lines)

    # Define the pattern to remove chapter headings and titles
    chapter_pattern = r'Chapter\s+[IVXLCDM]+\.\s*\n.*?(?=\n(?:Chapter\s+[IVXLCDM]+\.\s*|$))'

    # Remove chapter headings and titles
    text = re.sub(chapter_pattern, '', text)

    # Remove punctuation marks other than ".", "!", "?"
    text = re.sub(r'[^\w\s.!?]', '', text)

    


    # Remove newlines
    text = text.replace('\n', ' ')

    return text.strip()


# Example usage:
file_path = 'TheBrothersKaramazov.txt'
preprocessed_text = preprocess_text(file_path)


In [2]:
class n_gram_lang_model:
    def __init__(self):
        pass
    def preprocess_text(self,file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()[173:37246]  
            text = ''.join(lines)

        chapter_pattern = r'Chapter\s+[IVXLCDM]+\.\s*\n.*?(?=\n(?:Chapter\s+[IVXLCDM]+\.\s*|$))'

        text = re.sub(chapter_pattern, '', text)

        text = re.sub(r'[^\w\s.!?]', '', text)

    
        text = text.replace('\n', ' ')

        return text.strip()
    
    def add_sos_tokens(self,sentence):
        # Split the sentence into words
        words = sentence.split()

        # Iterate over the words and add three EOS tokens after '.', '?', or '!'
        updated_sentence = []
        for word in words:
            updated_sentence.append(word)
            if word.endswith('.') or word.endswith('?') or word.endswith('!'):
                updated_sentence.extend([ 'EOS'])

        return ' '.join(updated_sentence)

    def sentence_tokenize(self,text):
        
        sentences = nltk.sent_tokenize(text)
        sentences = [sentence.strip() for sentence in sentences]
        
        
        
        sentences_with_eos = []

        # Iterate through each sentence
        for i in range(len(sentences)):
            # Extract the current sentence
            current_sentence = sentences[i].strip()


            # Append the current sentence to the list
            if i == 0:
                current_sentence_with_eos = current_sentence + " <EOS>"
                sentences_with_eos.append(current_sentence_with_eos)

            # If it's not the last sentence
            if i < len(sentences) - 1:
                # Extract the last word of the current sentence
                last_word = current_sentence.split()[-1]
                sentences_with_eos.append(sentences[i + 1].strip()+ " <EOS>")
                next_sentence = sentences[i + 1].strip()

                # Concatenate the last word of the current sentence with the next sentence
                next_sentence_with_start_word = last_word + " " + next_sentence

                # Append the modified next sentence to the list
                sentences_with_eos.append(next_sentence_with_start_word.strip()+ " <EOS>")
        return sentences_with_eos




       
        
        
    def n_gram_extractor(self,n,sentences):
        Ngram = []
        self.n = n
        for sent in sentences:
            Ngram.extend(list(ngrams(sent.split(' '), self.n, pad_left=True, left_pad_symbol='SOS')))
        
        #text_n_grams= list(ngrams(text.split(' '), self.n,pad_left=True, left_pad_symbol='SOS'))
        #Ngram.extend(text_n_grams)
                
        return Ngram
    
    def model_training(self,ngram):
        freq_ngram = FreqDist(ngram)
        model = ConditionalFreqDist()
        for a, b, c,d in freq_ngram:
            model[(a,b,c)][d] += freq_ngram[a, b,c,d]

        #for phrase in ngram:
            #model[phrase[:self.n-1]][phrase[self.n-1]] += freq_ngram[phrase]
        return model
    
    def thresholding(self,model,thresh):
    
        prob_mat = ConditionalFreqDist()
        for keys,vals in model.items():
            total = sum(model[keys].values())
            for k,v in model[keys].items():
                if v > thresh:
                    prob_mat[keys][k] = v
                    
        return prob_mat
    
    def query_prediction(self,model, input_query,pred_len):
        
        sent = ' '.join(input_query)
        for i in range(pred_len):
            if (len(model[input_query])!=0) :
            
                suffix = pd.DataFrame([i[0] for i in model[input_query].most_common(5)]).sample(1).values[0][0]
                sent=sent+' '+ suffix

                input_query = sent.split(' ')[1:]
                input_query = tuple(input_query)
        if input_query[-1] == 'SOS':
            input_query
        query = sent.replace('SOS', '')
        
        
        return query
    

In [3]:
file_path = 'TheBrothersKaramazov.txt'
ngram_model = n_gram_lang_model()
preprocessed_text = ngram_model.preprocess_text(file_path)
sentences= ngram_model.sentence_tokenize(preprocessed_text)

#text = ngram_model.add_sos_tokens(preprocessed_text)

In [154]:
ngram = ngram_model.n_gram_extractor(4,sentences)

In [155]:
model = ngram_model.model_training(ngram)


In [156]:
prob_mat = ngram_model.thresholding(model,2000)


In [157]:
def update_input(input_query, prediction):
    predicted_words = prediction.split()

    updated_input = input_query[1:] + (predicted_words[-1],)
    return updated_input


In [182]:
start = 'this'
input_query = ('SOS','SOS','this')
text = []
num = 0
while num<50:    
    query = ngram_model.query_prediction(model, input_query,1)
    #print(" ".join(text))
    if query.split(' ')[-1] == '<EOS>':
        num +=1
        input_query = ('SOS','SOS',query.split(' ')[-2])
        
        print('sentence {}:'.format(num) , start+" "+" ".join(text))
        start = query.split(' ')[-2]
        text = []
        
        
    else:
            
        input_query = update_input(input_query, query)
        text.append(query.split()[-1])
        
    
    
        
    
    
    
    


sentence 1: this has so much time to spend on sentimentality with boys.
sentence 2: boys. They began teasing Ilusha at once.
sentence 3: once. The old man has begun to lock himself in at night and goes grouseshooting in the daytime and thats how he lives.
sentence 4: lives. Ive established myself in his room.
sentence 5: room. He suddenly pictured the garden the path behind the garden the door of the prison superintendent too who though scrupulous in the performance of his duties was a kind of presentiment of it I am sure some one told you so!
sentence 6: so! Why at Mokroe?
sentence 7: Mokroe? But its night!
sentence 8: night! Once the lad had all now the lad has naught cried Mitya suddenly.
sentence 9: suddenly. How can I forgive His tormentors?
sentence 10: tormentors? she bids all the saints all the martyrs all the angels and archangels to fall down with her and her unfinished answer left on the table he was a general burst of laughter.
sentence 11: laughter. He literally shook with